In [1]:
# !pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [1]:
import torch
torch.__version__

'1.10.0+cu113'

In [2]:
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
use_gpu = torch.cuda.is_available()
print(use_gpu)
if_gpu = torch.cuda.is_available()  # whether available
print(if_gpu)
gpu_number = torch.cuda.current_device()
print(gpu_number)

True
True
0


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import librosa.display
import torch
# import torchvision
# import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
import os

audio_path = "./Testing/14.  Boots Randolph - Yakety Sax-1.wav"


x, sr = librosa.load(audio_path)
print(f'x = {x}')
print(f'x.shape = {x.shape}')
print(f'sr = {sr}')


x = [ 0.05873846  0.05935711 -0.0090022  ... -0.1926996  -0.18709995
 -0.19382837]
x.shape = (441000,)
sr = 22050


In [5]:
with h5py.File('combine-22.h5', 'r') as f:
# with h5py.File('LPOD-22.h5', 'r') as f:
# with h5py.File('train_data_whole2.h5', 'r') as f:
    for key in f.keys():
        print(f[key].name)
        print(f[key].shape)

f.close()

/banjo
(74, 128, 22)
/bass clarinet
(545, 128, 22)
/bassoon
(313, 128, 22)
/cello
(660, 128, 22)
/clarinet
(906, 128, 22)
/contrabassoon
(605, 128, 22)
/cor anglais
(258, 128, 22)
/double bass
(622, 128, 22)
/flute
(452, 128, 22)
/french horn
(245, 128, 22)
/gac
(599, 128, 22)
/gel
(760, 128, 22)
/guitar
(29, 128, 22)
/mandolin
(60, 128, 22)
/oboe
(244, 128, 22)
/org
(676, 128, 22)
/percussion
(77, 128, 22)
/pia
(645, 128, 22)
/saxophone
(1107, 128, 22)
/trombone
(440, 128, 22)
/trumpet
(916, 128, 22)
/tuba
(555, 128, 22)
/viola
(147, 128, 22)
/violin
(1084, 128, 22)
/voi
(777, 128, 22)


In [11]:
# Open dataset
# keys = ['cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'tru', 'voi', 'sax', 'vio']
keys = ['banjo', 'bass clarinet', 'bassoon', 'cello', 'clarinet', 'contrabassoon', 'cor anglais', 'double bass', 'flute', 'french horn', 'guitar', 'mandolin', 'oboe', 'saxophone', 'trombone', 'trumpet', 'tuba', 'viola', 'violin']
# keys = ['banjo','cel','cla','flute','french_horn','gac','gel','guitar', 'mandolin', 'oboe','org','percussion','pia','saxophone', 'trombone', 'trumpet','tuba', 'viola', 'violin']
# dataset = h5py.File('LPOD-22.h5', 'r')
dataset = h5py.File('combine-22.h5', 'r')
# dataset = h5py.File('train_data_whole2.h5', 'r')

num_of_labels = len(keys)  # 2
num_of_tracks = sum([dataset[x].shape[0] for x in keys])  # 256

In [12]:
# Prepare data for training and testing
features = np.zeros((num_of_tracks, 128, 22), dtype=np.float32)   # (3, 128, 22)
labels = np.zeros((num_of_tracks, len(keys)), dtype=np.float32)   # (3, 2)

i = 0
for ki, k in enumerate(keys):
    features[i:i + len(dataset[k])] = np.nan_to_num(dataset[k])  # 使用0代替数组x中的nan元素
    labels[i:i + len(dataset[k]), ki] = 1
    i += len(dataset[k])

print(features.shape) # (2, 128, 22)
print(labels.shape)  # (2, 2)

# Split trainset to train and evaluation
# X_train,X_test, y_train, y_test = train_test_split(train_data,train_target,test_size=0.4, random_state=0)
X_train, X_eval, Y_train, Y_eval = train_test_split(features, labels, test_size=0.1, random_state=1337)
print(X_train.shape)
print(X_eval.shape)

(9262, 128, 22)
(9262, 19)
(8335, 128, 22)
(927, 128, 22)


In [24]:
# Prepare Pytorch dataloader
X_train_torch = torch.from_numpy(X_train).to(device)
X_eval_torch = torch.from_numpy(X_eval).to(device)
Y_train_torch = torch.from_numpy(Y_train).to(device)
Y_eval_torch = torch.from_numpy(Y_eval).to(device)

print(type(X_train_torch))

trainset = torch.utils.data.TensorDataset(X_train_torch, Y_train_torch)
evalset = torch.utils.data.TensorDataset(X_eval_torch, Y_eval_torch)

print(type(trainset))

train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=0)
eval_dataloader = torch.utils.data.DataLoader(evalset, batch_size=1, shuffle=True, num_workers=0)

<class 'torch.Tensor'>
<class 'torch.utils.data.dataset.TensorDataset'>


In [ ]:
# Han16
class Han16(nn.Module):
	def __init__(self):
		super().__init__()
        
		self.conv1_1 = nn.Conv2d(1, 32, 3, padding=2)
		self.conv1_2 = nn.Conv2d(32, 32, 3, padding=2)
		self.conv2_1 = nn.Conv2d(32, 64, 3, padding=2)
		self.conv2_2 = nn.Conv2d(64, 64, 3, padding=2)
		self.conv3_1 = nn.Conv2d(64, 128, 3, padding=2)
		self.conv3_2 = nn.Conv2d(128, 128, 3, padding=2)
		self.conv4_1 = nn.Conv2d(128, 256, 3, padding=2)
		self.conv4_2 = nn.Conv2d(256, 256, 3, padding=2)
		self.pool = nn.MaxPool2d(3, stride=3)
		self.zero_pad = nn.ZeroPad2d(1)
		# self.pool1 = nn.MaxPool2d(x.size(dim=2), x.size(dim=3))
		self.fc1 = nn.Linear(256, 1024)
		self.fc_output = nn.Linear(1024, num_of_labels)
                
		for m in self.modules():
			if isinstance(m, nn.Conv2d):
				nn.init.xavier_normal_(m.weight)
				nn.init.constant(m.bias, 0)

	def forward(self, x):
		x = F.leaky_relu(self.conv1_1(self.zero_pad(x)))
		x = self.pool(F.leaky_relu(self.conv1_2(self.zero_pad(x)), negative_slope=0.33))
		x = F.dropout(x, p=0.25)
		x = F.leaky_relu(self.conv2_1(self.zero_pad(x)))
		x = self.pool(F.leaky_relu(self.conv2_2(self.zero_pad(x)), negative_slope=0.33))
		x = F.dropout(x, p=0.25)
		x = F.leaky_relu(self.conv3_1(self.zero_pad(x)))
		x = self.pool(F.leaky_relu(self.conv3_2(self.zero_pad(x)), negative_slope=0.33))
		x = F.dropout(x, p=0.25)
		x = F.leaky_relu(self.conv4_1(self.zero_pad(x)))
		x = F.leaky_relu(self.conv4_2(self.zero_pad(x)), negative_slope=0.33)
		x = F.max_pool2d(x, kernel_size=x.size()[2:]) # global max pooling
		x = x.view(-1, 256)
		x = F.leaky_relu(self.fc1(x), negative_slope=0.33)
		x = F.dropout(x, p=0.5)
		x = self.fc_output(F.sigmoid(x))
		return x

In [26]:
# ConvBlocks + bilstm + Transformer
class ConvBlocks(nn.Module):
    def __init__(self,):
        super(ConvBlocks, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(128),
            nn.Conv2d(128,128, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.Conv2d(256,256, kernel_size=(3, 3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),
            nn.ReLU(),
            
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.Conv2d(512,512, kernel_size=(3, 3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=(2,1), stride=(2,1)),
            nn.ReLU(),
            
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.Conv2d(512,512, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=(2,1), stride=(2,1)),
            nn.ReLU(),
            
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.Conv2d(512,512, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1),padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(kernel_size=(2,1), stride=(2,1)),
            nn.ReLU(),
            

        )
        
    def forward(self,inputs):
        out = self.conv(inputs)
        out = out.flatten(start_dim=1, end_dim=2)
        return out
   

 
class Model(nn.Module):
    def __init__(self,num_classes=9):
        super(Model, self).__init__()
        self.num_classes = num_classes
        
        self.conv = ConvBlocks()
        self.blstm = nn.LSTM(1024, hidden_size=int(320/2),bidirectional=True, batch_first=True)
#         self.mha = TransformerEncoderLayer(embed_dim=320, num_heads=10,temp=0.2)
        self.fc1 = nn.Linear(320, 512)
        self.fc2 = nn.Linear(512, self.num_classes)
        self.fc3 = nn.Linear(1024, 320)
        self.fc4 = nn.Linear(512, 320)
    
    def forward(self, inputs):
        cnn_out = self.conv(inputs)
        cnn_out = cnn_out.permute(0,2,1)   

        # bilstm layer
        rnn_out,_ = self.blstm(cnn_out)
        rnn_out = rnn_out.permute(1,0,2)
#         print(rnn_out.shape)

        
#      # Transformer layer
        rnn_out = self.fc3(cnn_out)   
#         mha_out = self.mha(rnn_out)
#         mha_out = mha_out.permute(1,0,2)
        
#         print(mha_out.shape)

        
        pooled = torch.mean(rnn_out, dim=1)
        fc1_out = self.fc1(pooled)
        out = self.fc2(fc1_out)
        out = torch.sigmoid(out)
#         print(out.shape) # torch.Size([1, 11])
        
        return out
        

In [28]:
# Initialisation
# net = Net2()
# net = ConvBlocks()
# net = Model(19).to(device)
net = Han16().to(device)
criterion = nn.CrossEntropyLoss()

# optimiser = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
optimiser = optim.Adam(net.parameters(), lr=0.001)
for epoch in range(25): 
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_dataloader, start=0):

        inputs = inputs.unsqueeze(1) # add one dimension # torch.Size([1, 1, 128, 22])

        optimiser.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs) # torch.Size([1, 5])
        _, labels = torch.max(labels, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()


        # print statistics
        running_loss += loss.item()
        if i == len(train_dataloader)-1:  # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
print('Finished Training')
PATH = './wave_model_han16net.pth'
# PATH = './wave_model_net.pth'
torch.save(net.state_dict(), PATH)

c:\users\neko_maru~\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


[1,  5829] loss: 173.523
[2,  5829] loss: 172.710
[3,  5829] loss: 171.631
[4,  5829] loss: 171.631
[5,  5829] loss: 171.631
[6,  5829] loss: 171.631
[7,  5829] loss: 171.631
[8,  5829] loss: 171.631
[9,  5829] loss: 171.631
[10,  5829] loss: 171.631
[11,  5829] loss: 171.631
[12,  5829] loss: 171.631
[13,  5829] loss: 171.631
[14,  5829] loss: 171.631
[15,  5829] loss: 171.631
[16,  5829] loss: 171.631
[17,  5829] loss: 171.631


KeyboardInterrupt: 

In [ ]:
net.load_state_dict(torch.load(PATH))  # 将预训练的参数权重加载到新的模型之中

correct = 0
total = 0
with torch.no_grad():
    for i, (inputs, labels) in enumerate(eval_dataloader, start=0):
        inputs = inputs.unsqueeze(1) # add one dimension
        outputs = net(inputs)
        newlabels = labels > 0
        indices =  newlabels.nonzero()     
        print(outputs.data)
        pred_sum = outputs.data.sum()
        _, pred_max = torch.max(outputs.data, 1)
        pred_max = outputs.data[0][pred_max]
        print(pred_sum)
        print(pred_max)
        pred_index = outputs.data / pred_max
        print(1)
        print(pred_index)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print(indices)
        correct += (predicted == indices[0][1]).sum().item()
        print('label:')
        print(labels)
        print('newlabel:')
        print(indices[0][1])
        print('predicted:')
        print(predicted)        
        print('Total: %d' % total)
        print('Correct: %d' % correct)
        

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))